In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sksurv.nonparametric import kaplan_meier_estimator
from sksurv.linear_model import CoxPHSurvivalAnalysis
from lifelines import CoxPHFitter
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("survival.csv")

In [3]:
df.rename(columns={'Vibration - X':'VibrationX','Vibration  Y':'VibrationY','Vibration  Z':'VibrationZ'},inplace=True)

In [4]:
columns = ['Lifetime', 'Broken', 'Temperature', 'Voltage', 'Current', 'Humidity','VibrationX', 'VibrationY', 'VibrationZ']
data = df.loc[:,columns]

In [5]:
coxph = CoxPHFitter()
coxph.fit(data, duration_col='Lifetime', event_col='Broken')

<lifelines.CoxPHFitter: fitted with 1255 total observations, 478 right-censored observations>

In [7]:
def predict(a,c,d,e,f,g,h,i):
    data1 = pd.DataFrame({'Lifetime':[a],'Temperature':[c],'Voltage':[d],'Current':[e],'Humidity':[f],
                          'VibrationX':[g],'VibrationY':[h],'VibrationZ':[i]})
    pred_prob = coxph.predict_survival_function(data1,conditional_after=data1['Lifetime'])
    
    lifetime1 = []
    for i in pred_prob.columns:
        lifetime1.append(pred_prob.index[pred_prob[i]==pred_prob[i].quantile(0.5)][0])
        
    result1 = pd.DataFrame({'pred_lifetime':lifetime1})
    output1 = pd.merge(data1,result1,left_index=True, right_index=True)
    output1['RUL'] = output1.pred_lifetime - output1.Lifetime
    #print(output1['pred_lifetime'][0])
    #print(output1['RUL'][0])
    print(output1.head())

In [ ]:
predict(60,30,230,1.5,36,0.494,1.290,9.120)

In [ ]:
jgjkhjl

In [ ]:
import pickle

In [ ]:
with open('RUL_Cox.pkl', 'wb') as files:
    pickle.dump(coxph, files)

In [ ]:
with open('RUL_Cox.pkl' , 'rb') as f:
    RUL_Cox = pickle.load(f)

In [ ]:
from flask import Flask, render_template, request
app = Flask(__name__)
Bdt = pickle.load(open("RUL_Cox.pkl",'rb'))
@app.route("/")
def home():
    return render_template('RUL_machine.html')
@app.route("/predict", methods=['GET','POST'])
def predict():
    if request.method == 'POST':
        #access the data from form

        Lifetime = int(request.form["Lifetime"])
        Broken = int(request.form["Broken"])
        Temperature = float(request.form["Temperature"])
        Voltage = float(request.form["Voltage"])
        Current = float(request.form["Current"])
        Humidity = float(request.form["Humidity"])
        VibrationX = float(request.form["VibrationX"])
        VibrationY = float(request.form["VibrationY"])
        VibrationZ = float(request.form["VibrationZ"])
        
        
        
        #get prediction
        def predict(a,b,c,d,e,f,g,h,i):
            data1 = pd.DataFrame({'Lifetime':[a],'Broken':[b],'Temperature':[c],'Voltage':[d],'Current':[e],'Humidity':[f],
                          'VibrationX':[g],'VibrationY':[h],'VibrationZ':[i]})
            pred_prob = coxph.predict_survival_function(data1,conditional_after=data1['Lifetime'])
    
            lifetime1 = []
            for i in pred_prob.columns:
                lifetime1.append(pred_prob.index[pred_prob[i]==pred_prob[i].quantile(0.5)][0])
        
            result1 = pd.DataFrame({'pred_lifetime':lifetime1})
            output1 = pd.merge(data1,result1,left_index=True, right_index=True)
            output1['RUL'] = output1.pred_lifetime - output1.Lifetime
            output = output1['RUL'][0]
        return render_template("RUL_machine.html", prediction_text='Your predicted Remaining Lifetime of machine is  {}'.format(output))
if __name__ == "__main__":
    app.run(debug=True)